This code will generate the feature vector:
)


    


3. Loop through the days again and add the derived features (Fuel moisture, KBDI-FWI, fbp, RH, Haines, HDW)

In [71]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

##  Step 0. Throw out fires that have 0 viirs detects on any day. (fires are already filtered to be in the right lat/lon range

In [2]:
# load in the polygons and attributes
fire_polygons = gpd.read_file('fire_polygons_only.geojson')
fire_attributes = pd.read_csv('fire_polygons_attributes.csv')
incidents = pd.read_csv('unique_fires.csv')
print(len(fire_polygons), len(fire_attributes), len(incidents))

#throw out fires that have 0 days with detects
daily_drop_inds = []
incident_drop_inds = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    if len(pts[pts==0])==len(pts):
        #print(item, 'oops all zeros')
        daily_drop_inds = np.append(daily_drop_inds, np.where(fire_attributes['Incident Number']==item)[0])
        incident_drop_inds = np.append(incident_drop_inds, np.where(incidents['Incident Number']==item)[0])

daily_drop_inds = [int(p) for p in daily_drop_inds]
#print(daily_drop_inds)

fire_attributes = fire_attributes.drop(labels= daily_drop_inds)
fire_polygons = fire_polygons.drop(labels = daily_drop_inds)
incidents = incidents.drop(labels = incident_drop_inds)
print(len(fire_polygons), len(fire_attributes), len(incidents))
fire_attributes

22280 22280 1354
19137 19137 1036


,Unnamed: 0,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points
5,5,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0
6,6,10662684.0,PAINTED WAGON,2019-04-09,33.786944,-112.753333,0.0
7,7,10662684.0,PAINTED WAGON,2019-04-10,33.786944,-112.753333,0.0
8,8,10663171.0,LONE MOUNTAIN,2019-05-26,33.808056,-105.738611,0.0
9,9,10663171.0,LONE MOUNTAIN,2019-05-27,33.808056,-105.738611,0.0
...,...,...,...,...,...,...,...
22275,22275,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0
22276,22276,11980587.0,CREEK 5,2020-12-26,33.402646,-117.266795,0.0
22277,22277,11980587.0,CREEK 5,2020-12-27,33.402646,-117.266795,0.0
22278,22278,11980587.0,CREEK 5,2020-12-28,33.402646,-117.266795,0.0


## 1. Figure out the start and end date we will use for each fire

In [ ]:
# first check what fraction of the fire lifetimes are 0 days
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    #print(len(np.where(pts==0)[0]))
    print('The fire lifetime is ', len(np.where(pts!=0)[0])*100/len(pts), '% nonzero detections')

In [3]:
keep_days = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    dys = fire_attributes['Current Day'].values[fire_attributes['Incident Number']==item]
    nonzero = np.where(pts!=0)[0]
    first_day = dys[nonzero[0]]
    last_day = dys[nonzero[len(nonzero)-1]]
    keep_days = np.append(keep_days, np.where((fire_attributes['Incident Number']==item)&(fire_attributes['Current Day']>=first_day)&(fire_attributes['Current Day']<=last_day))[0])

keep_days = [int(p) for p in keep_days]
print(len(keep_days))
fire_attributes = fire_attributes.iloc[keep_days, :]
fire_polygons = fire_polygons.iloc[keep_days, :]

fire_polygons

8161


,geometry
5,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
22,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
23,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
24,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
25,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...
22271,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
22272,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
22274,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
22275,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


## 2. Loop through the days and get the raw features (temp at different pressure levels, wind, pressure, precip, terrain info) and the labels (QFED PM2.5 from the next day)

In [6]:
# THIS WAY IS PRETTY SLOW

#pre-allocate the data frame
num_days = len(fire_attributes)
fire_features = gpd.GeoDataFrame({'Incident Number': np.nan*np.zeros(num_days), 
                          'Fire Name': np.nan*np.zeros(num_days), 
                         'Current Day': np.nan*np.zeros(num_days),
                        'Lat Fire': np.nan*np.zeros(num_days), 
                     'Lon Fire': np.nan*np.zeros(num_days), 
                         'Number of VIIRS points': np.nan*np.zeros(num_days),
                        'TLML': np.nan*np.zeros(num_days), 
                        'QLML': np.nan*np.zeros(num_days),
                                  'SPEEDLML': np.nan*np.zeros(num_days),
                                  'PS': np.nan*np.zeros(num_days),
            'geometry': np.nan*np.zeros(num_days)}) # pre-allocate the dataframe

fire_features['Incident Number'] = fire_attributes['Incident Number'].values
fire_features['Fire Name'] = fire_attributes['Fire Name'].values
fire_features['Current Day'] = fire_attributes['Current Day'].values
fire_features['Lat Fire'] = fire_attributes['Lat Fire'].values
fire_features['Lon Fire'] = fire_attributes['Lon Fire'].values
fire_features['Number of VIIRS points'] = fire_attributes['Number of VIIRS points'].values
fire_features['geometry'] = fire_polygons['geometry'].values

varis = {'WESTUS_MERRA2_400.inst1_2d_lfo_Nx.':['TLML', 'QLML', 'SPEEDLML', 'PS']}
locs = {'WESTUS_MERRA2_400.inst1_2d_lfo_Nx.':['lat', 'lon']}
subfolders = {'/MERRA2/'}
file_suffix = '.nc4'

for ii in range(num_days): #loop over all the days
    #get the day
    year = fire_features['Current Day'][ii][0:4]
    month = fire_features['Current Day'][ii][5:7]
    day = fire_features['Current Day'][ii][8:10]
    #print(year, month, day)
    
    for folder in subfolders:
        path = '/data2/lthapa/'+str(year)+folder
        #print(path)
        
        #THIS COULD BE A SEPARATE HELPER FUNCTION
        for file_prefix in varis.keys(): # for each dataset
            path = path+file_prefix+year+month+day+file_suffix
            print(path)
            dataset = nc.Dataset(path)
            
            #get the lat and lon for the dataset
            loc_vals = []
            for loc in locs.values():
                loc_vals.append(loc)
            loc_vals = loc_vals[0]
            lat = dataset[loc_vals[0]][:]
            lon = dataset[loc_vals[1]][:]
            #turn it into a meshgrid
            LON, LAT = np.meshgrid(lon, lat)
            #get the full variable, put it to daily resolution
            for value in varis.values(): 
                for var in value:
                    variable = dataset[var][:]
                    variable = np.nanmean(variable, axis=0) #average across the day
                    #print(variable.shape, LON.shape, LAT.shape)
            
                    #find the intersection
                    overlaps = calculate_overlaps(LAT, LON, fire_features['geometry'][ii])
                    variable_weighted = np.multiply(variable,overlaps)
                    #save it to the dataframe
                    fire_features.loc[ii, var] = np.nansum(variable_weighted)
fire_features

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190408.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190609.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190610.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190611.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190612.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190613.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190614.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190615.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190616.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190617.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190618.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190619.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190620.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190721.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190715.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190525.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190713.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190714.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190715.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190716.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190717.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190718.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190719.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190720.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190721.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190722.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190723.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190723.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190723.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190803.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190804.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190805.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190806.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190807.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190808.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190809.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190810.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190811.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190812.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190827.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190828.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190829.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190830.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190831.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190901.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190902.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190903.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190904.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190905.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191028.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190803.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191024.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190819.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190820.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190821.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190822.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190823.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190824.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190825.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190826.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190827.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190828.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190829.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190830.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190831.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191003.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191004.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191005.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191006.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191007.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191008.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191009.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191010.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191011.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191012.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191013.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190904.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190905.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190705.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190706.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190707.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190708.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190709.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190710.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190711.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190712.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190713.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190624.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190820.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191007.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191008.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190912.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190913.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190914.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190915.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190916.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190802.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190803.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190804.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190805.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190806.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190807.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190808.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190809.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190901.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190902.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190903.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190904.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190905.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190820.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190816.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190817.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190818.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190819.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190820.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190821.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190822.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

KeyboardInterrupt: 

In [79]:
# dictionaries that track the variables, dimensions, times and levels
varis = {'/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.*.nc4':['TLML', 'QLML', 'SPEEDLML', 'PS'], 
        '/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.*.nc4': ['T', 'QV'], 
        '/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4':['biomass']}

#0=lat, 1=lon, 2=time, 3=level 
dims = {'/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.*.nc4':['lat', 'lon', 'time'], 
       '/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.*.nc4':['lat', 'lon', 'time', 'lev'],
       '/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4':['lat', 'lon', 'time']}
times = {'TLML':12, 'QLML':12, 'SPEEDLML':12, 'PS':12, 'T':12, 'QV':12, 'biomass':12}
levels = {'T':[500, 700], 'QV':[700]} 
file_suffix = '.nc4'

#pre-allocate the data frame
num_days = len(fire_attributes)
fire_features = gpd.GeoDataFrame({'Incident Number': np.nan*np.zeros(num_days), 'Fire Name': np.nan*np.zeros(num_days), 
                             'Current Day': np.nan*np.zeros(num_days), 'Lat Fire': np.nan*np.zeros(num_days), 
                             'Lon Fire': np.nan*np.zeros(num_days), 'Number of VIIRS points': np.nan*np.zeros(num_days),
                            'TLML_12Z': np.nan*np.zeros(num_days), 'QLML_12Z': np.nan*np.zeros(num_days),
                            'SPEEDLML_12Z': np.nan*np.zeros(num_days),'PS_12Z': np.nan*np.zeros(num_days),
                            'T_12Z_700mb': np.nan*np.zeros(num_days),'T_12Z_500mb': np.nan*np.zeros(num_days),
                            'QV_12Z_700mb': np.nan*np.zeros(num_days), 'biomass_12Z': np.nan*np.zeros(num_days),   
                            'geometry': np.nan*np.zeros(num_days)}) # pre-allocate the dataframe
#'fccs': np.nan*np.zeros(num_days),'slp': np.nan*np.zeros(num_days),'asp': np.nan*np.zeros(num_days), 
fire_features['Incident Number'] = fire_attributes['Incident Number'].values
fire_features['Fire Name'] = fire_attributes['Fire Name'].values
fire_features['Current Day'] = fire_attributes['Current Day'].values
fire_features['Lat Fire'] = fire_attributes['Lat Fire'].values
fire_features['Lon Fire'] = fire_attributes['Lon Fire'].values
fire_features['Number of VIIRS points'] = fire_attributes['Number of VIIRS points'].values
fire_features['geometry'] = fire_polygons['geometry'].values

#print(fire_features)
unique_days = np.unique(fire_features['Current Day'].values)
print(len(unique_days))
for dd in range(len(unique_days)):

    date=unique_days[dd]
    yr = date[0:4]
    mo = date[5:7]
    dy = date[8:10]
    print(yr, mo,dy)
    #find the polygons for the given day

    for i in varis : #loop over the file names
        #get the path to the file
        path = '/data2/lthapa/'+yr+i
        if '*' in path:
            path = path.replace('*', yr+mo+dy)
        if np.datetime64(date)>=np.datetime64('2020-09-01') and np.datetime64(date)<=np.datetime64('2020-09-30'):
            path = path.replace('400', '401')
            
        #load the file in as a netcdf dataset
        print(path)
        dat = nc.Dataset(path)
        ndims = len(dims[i])
        if ndims ==2: #lat and lon
            lat = dat[dims[i][0]][:]
            lon = dat[dims[i][1]][:]
        elif ndims == 3: #lat, lon, and time
            lat = dat[dims[i][0]][:]
            lon = dat[dims[i][1]][:]
            t = dat[dims[i][2]][:]
        elif ndims ==4: #lat,lon,time,and level
            lat = dat[dims[i][0]][:]
            lon = dat[dims[i][1]][:]
            t = dat[dims[i][2]][:]
            lev = dat[dims[i][3]][:]
        if len(lat.shape)==1:
            lon, lat = np.meshgrid(lon, lat)
        #calculate the areas of the lat/lon grid for the DATASET
        lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon)
        inds = np.where(fire_features['Current Day']==date)[0]
        polys = fire_features['geometry'].iloc[inds].values
        for p in range(len(inds)): #loop over the polygons
            poly = polys[p]
            tic = time.time()
            overlaps = calculate_overlaps(lat,lon,lat_corners,lon_corners, poly) #calculate the overlap between the poly and the lat/lon grid
            toc = time.time()
            print('Time elapsed for calculating overlaps for polygon:', toc-tic)
            for j in varis[i]: #loop over the variable names, which are lists
                var = dat[j][:]
                #print('the dimensions of', j, 'are', var.shape, 'and', j, 'has', ndims, 'dimensions')
                #select the times and levels we need
                if ndims ==2: #lat and lon
                    # calculate overlap
                    #print(var.shape)
                    var_weighted = np.multiply(var,overlaps)
                    var_ave = np.nansum(var_weighted)
                    name_in_table = var
                    fire_features.loc[inds[p],name_in_table] = var_ave
                elif ndims == 3: #lat, lon, and time
                    #select the needed times (12Z)
                    if j in times.keys():
                        ind_time = np.argmin(np.abs(times[j]*60-t))
                        #filter the variable by time
                        var = var[ind_time, :, :]
                        #print(var.shape)
                        var_weighted = np.multiply(var,overlaps)
                        var_ave = np.nansum(var_weighted)
                        name_in_table = j+'_'+str(times[j])+'Z'
                        fire_features.loc[inds[p],name_in_table] = var_ave

                elif ndims ==4: #lat,lon,time,and level
                    #select the needed times and levels
                    if j in times.keys():
                        ind_time = np.argmin(np.abs(times[j]*60-t))
                        #filter the variable by time
                        var = var[ind_time, :, :, :]
                        #filter by levels
                        if j in levels.keys(): #if there are levels, loop through them
                            for l in levels[j]:
                                ind_lev = np.argmin(np.abs(l-lev))
                                var_weighted = np.multiply(var[ind_lev, :, :], overlaps)
                                var_ave = np.nansum(var_weighted)
                                name_in_table = j+'_'+str(times[j])+'Z_'+str(l)+'mb'
                                fire_features.loc[inds[p],name_in_table] = var_ave
                
            
        dat.close() 
    
fire_features

442
2019 02 17
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190217.nc4
Time elapsed for calculating overlaps for polygon: 0.06892132759094238
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190217.nc4
Time elapsed for calculating overlaps for polygon: 0.06969738006591797
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190217.nc4
Time elapsed for calculating overlaps for polygon: 2.2317276000976562
2019 02 18
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190218.nc4
Time elapsed for calculating overlaps for polygon: 0.06878066062927246
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190218.nc4
Time elapsed for calculating overlaps for polygon: 0.06854796409606934
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190218.nc4
Time elapsed for calculating overlaps for polygon: 2.242173433303833
2019 02 24
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190224.nc4
Time elapsed for calculating overlaps for polyg

Time elapsed for calculating overlaps for polygon: 0.07221007347106934
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190505.nc4
Time elapsed for calculating overlaps for polygon: 0.06858420372009277
Time elapsed for calculating overlaps for polygon: 0.0692451000213623
Time elapsed for calculating overlaps for polygon: 0.07325220108032227
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190505.nc4
Time elapsed for calculating overlaps for polygon: 2.237149238586426
Time elapsed for calculating overlaps for polygon: 2.2410995960235596
Time elapsed for calculating overlaps for polygon: 2.3644049167633057
2019 05 06
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190506.nc4
Time elapsed for calculating overlaps for polygon: 0.06828069686889648
Time elapsed for calculating overlaps for polygon: 0.06868457794189453
Time elapsed for calculating overlaps for polygon: 0.06894731521606445
Time elapsed for calculating overlaps for polygon: 0.06964397430419922
Tim

Time elapsed for calculating overlaps for polygon: 0.06842565536499023
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190525.nc4
Time elapsed for calculating overlaps for polygon: 2.2352023124694824
Time elapsed for calculating overlaps for polygon: 2.2428996562957764
2019 05 26
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190526.nc4
Time elapsed for calculating overlaps for polygon: 0.06810832023620605
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190526.nc4
Time elapsed for calculating overlaps for polygon: 0.06833100318908691
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190526.nc4
Time elapsed for calculating overlaps for polygon: 2.2118988037109375
2019 05 27
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190527.nc4
Time elapsed for calculating overlaps for polygon: 0.06780886650085449
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190527.nc4
Time elapsed for calculating overlaps for polygon: 0.06825542449

Time elapsed for calculating overlaps for polygon: 0.06916379928588867
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190605.nc4
Time elapsed for calculating overlaps for polygon: 2.232637882232666
Time elapsed for calculating overlaps for polygon: 2.2330322265625
Time elapsed for calculating overlaps for polygon: 2.225609540939331
Time elapsed for calculating overlaps for polygon: 2.2202067375183105
Time elapsed for calculating overlaps for polygon: 2.233964443206787
2019 06 06
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190606.nc4
Time elapsed for calculating overlaps for polygon: 0.07253313064575195
Time elapsed for calculating overlaps for polygon: 0.06890034675598145
Time elapsed for calculating overlaps for polygon: 0.06890439987182617
Time elapsed for calculating overlaps for polygon: 0.06834983825683594
Time elapsed for calculating overlaps for polygon: 0.06897854804992676
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190606.nc4
Time elap

Time elapsed for calculating overlaps for polygon: 0.06841373443603516
Time elapsed for calculating overlaps for polygon: 0.06865429878234863
Time elapsed for calculating overlaps for polygon: 0.06870198249816895
Time elapsed for calculating overlaps for polygon: 0.06979250907897949
Time elapsed for calculating overlaps for polygon: 0.07044792175292969
Time elapsed for calculating overlaps for polygon: 0.06840848922729492
Time elapsed for calculating overlaps for polygon: 0.06986474990844727
Time elapsed for calculating overlaps for polygon: 0.06847548484802246
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190610.nc4
Time elapsed for calculating overlaps for polygon: 0.0685265064239502
Time elapsed for calculating overlaps for polygon: 0.0684354305267334
Time elapsed for calculating overlaps for polygon: 0.06870174407958984
Time elapsed for calculating overlaps for polygon: 0.0681612491607666
Time elapsed for calculating overlaps for polygon: 0.06933021545410156
Time el

...

Time elapsed for calculating overlaps for polygon: 2.2425854206085205
2020 11 10
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201110.nc4
Time elapsed for calculating overlaps for polygon: 0.06857442855834961
Time elapsed for calculating overlaps for polygon: 0.06879949569702148
Time elapsed for calculating overlaps for polygon: 0.06859660148620605
Time elapsed for calculating overlaps for polygon: 0.06881308555603027
Time elapsed for calculating overlaps for polygon: 0.06972050666809082
Time elapsed for calculating overlaps for polygon: 0.06952285766601562
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201110.nc4
Time elapsed for calculating overlaps for polygon: 0.06798553466796875
Time elapsed for calculating overlaps for polygon: 0.06856989860534668
Time elapsed for calculating overlaps for polygon: 0.06850576400756836
Time elapsed for calculating overlaps for polygon: 0.06899809837341309
Time elapsed for calculating overlaps for polygon: 0.0694839954

Time elapsed for calculating overlaps for polygon: 2.250570058822632
Time elapsed for calculating overlaps for polygon: 2.2354698181152344
2020 11 16
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201116.nc4
Time elapsed for calculating overlaps for polygon: 0.06831765174865723
Time elapsed for calculating overlaps for polygon: 0.0684058666229248
Time elapsed for calculating overlaps for polygon: 0.0688323974609375
Time elapsed for calculating overlaps for polygon: 0.06949853897094727
Time elapsed for calculating overlaps for polygon: 0.06942129135131836
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201116.nc4
Time elapsed for calculating overlaps for polygon: 0.06790280342102051
Time elapsed for calculating overlaps for polygon: 0.06836652755737305
Time elapsed for calculating overlaps for polygon: 0.06881928443908691
Time elapsed for calculating overlaps for polygon: 0.06895852088928223
Time elapsed for calculating overlaps for polygon: 0.06920957565307

Time elapsed for calculating overlaps for polygon: 0.06975364685058594
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201123.nc4
Time elapsed for calculating overlaps for polygon: 0.06804156303405762
Time elapsed for calculating overlaps for polygon: 0.06879711151123047
Time elapsed for calculating overlaps for polygon: 0.06928014755249023
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201123.nc4
Time elapsed for calculating overlaps for polygon: 2.2209572792053223
Time elapsed for calculating overlaps for polygon: 2.2419700622558594
Time elapsed for calculating overlaps for polygon: 2.2506487369537354
2020 11 24
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201124.nc4
Time elapsed for calculating overlaps for polygon: 0.06842231750488281
Time elapsed for calculating overlaps for polygon: 0.06885552406311035
Time elapsed for calculating overlaps for polygon: 0.07021450996398926
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201124.nc4

Time elapsed for calculating overlaps for polygon: 0.0685110092163086
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201204.nc4
Time elapsed for calculating overlaps for polygon: 0.06765580177307129
Time elapsed for calculating overlaps for polygon: 0.0689702033996582
Time elapsed for calculating overlaps for polygon: 0.06824851036071777
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201204.nc4
Time elapsed for calculating overlaps for polygon: 2.2216544151306152
Time elapsed for calculating overlaps for polygon: 2.265678882598877
Time elapsed for calculating overlaps for polygon: 2.2664248943328857
2020 12 05
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201205.nc4
Time elapsed for calculating overlaps for polygon: 0.06751608848571777
Time elapsed for calculating overlaps for polygon: 0.06879711151123047
Time elapsed for calculating overlaps for polygon: 0.06885981559753418
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201205.nc4
Ti

,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,TLML_12Z,QLML_12Z,SPEEDLML_12Z,PS_12Z,T_12Z_700mb,T_12Z_500mb,QV_12Z_700mb,biomass_12Z,geometry
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,290.748108,0.004608,10.770707,91644.742188,281.218140,261.982361,0.001890,7.627477e-10,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,290.003723,0.003563,8.092171,81498.414062,286.447601,266.682251,0.002318,0.000000e+00,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,281.681793,0.006098,10.386342,82444.875000,281.240356,262.869751,0.007128,0.000000e+00,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,283.622253,0.009279,6.614388,81877.890625,283.966187,263.090515,0.005877,0.000000e+00,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,284.536926,0.006591,12.682461,81796.609375,284.776917,267.068909,0.004644,0.000000e+00,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,293.930328,0.003627,7.339819,91107.664062,282.576782,262.727051,0.004921,0.000000e+00,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
8157,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,279.254852,0.001604,6.979411,90997.539062,276.615021,257.887451,0.001588,1.553367e-09,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
8158,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,288.125366,0.001511,10.678336,99387.382812,275.902649,258.410187,0.000153,0.000000e+00,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
8159,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,287.946075,0.002882,5.147175,99128.710938,272.194824,258.612366,0.002691,2.873148e-10,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


In [81]:
therest = fire_features[['Incident Number', 'Fire Name', 'Current Day','Lat Fire', 'Lon Fire','Number of VIIRS points', 'TLML_12Z', 'QLML_12Z','SPEEDLML_12Z','PS_12Z','T_12Z_700mb','T_12Z_500mb','QV_12Z_700mb', 'biomass_12Z']]



therest.to_csv('fire_features.csv')

## Helper Functions

In [69]:
#this function calculates the area of each grid in a given grid cell
# LAT and LON should be 2d
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LAT[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    lon_corners = (LON[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LON[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    return lat_corners, lon_corners
    """for ii in range(1,LAT.shape[0]-1):
        for jj in range(1, LAT.shape[1]-1):
            cell_lat = LAT[ii, jj]
            cell_lon = LON[ii, jj]
            SW_lat[ii,jj] = LAT[ii,jj] + (LAT[ii-1,jj-1] - LAT[ii,jj])/2
            SW_lon[ii,jj] = LON[ii,jj] + (LON[ii-1,jj-1] - LON[ii,jj])/2
            SE_lat[ii,jj] = LAT[ii,jj] + (LAT[ii-1,jj-1] - LAT[ii,jj])/2
            SE_lon[ii,jj] = LON[ii,jj] + (LON[ii,jj] - LON[ii-1,jj-1])/2
            NW_lat[ii,jj] = LAT[ii,jj] + (LAT[ii,jj] - LAT[ii-1,jj-1])/2
            NW_lon[ii,jj] = LON[ii,jj] + (LON[ii-1,jj-1] - LON[ii,jj])/2
            NE_lat[ii,jj] = LAT[ii,jj] + (LAT[ii,jj] - LAT[ii-1,jj-1])/2
            NE_lon[ii,jj] = LON[ii,jj] + (LON[ii,jj] - LON[ii-1,jj-1])/2
    return  SW_lat, SW_lon, SE_lat, SE_lon, NW_lat, NW_lon, NE_lat, NE_lon"""

In [74]:
# this function calculates the overlap between the given shape (fire_shape) and given grid (LAT, LON). 
#LAT and LON should be 2D

def calculate_overlaps(lat, lon, lat_corners, lon_corners, fire_shape):
    overlaps = np.zeros(lat.shape)
    # buffer the fire shape so it has a non zero area
    if fire_shape.area ==0:
        fire_shape = fire_shape.buffer(0.01)
    # find the intersection between the first day's polygon and the  grid
    for ii in range(1,lat.shape[0]-1):
        for jj in range(1, lat.shape[1]-1):
            #print('ctr', lat[ii,jj], lon[ii,jj])
            """ SW_lat = lat_corners[ii-1, jj-1]
            SW_lon = lon_corners[ii-1, jj-1]
            SE_lat = lat_corners[ii-1, jj-1]
            SE_lon = lon_corners[ii-1, jj]
            NW_lat = lat_corners[ii, jj-1]
            NW_lon = lon_corners[ii, jj-1]
            NE_lat = lat_corners[ii, jj]
            NE_lon = lon_corners[ii, jj]"""
            poly_cell = Polygon([(lon_corners[ii, jj-1],lat_corners[ii, jj-1]), (lon_corners[ii, jj],lat_corners[ii, jj]), (lon_corners[ii-1, jj],lat_corners[ii-1, jj-1]), (lon_corners[ii-1, jj-1], lat_corners[ii-1, jj-1])])
            overlap_cell = fire_shape.intersection(poly_cell)
            #overlaps are the fraction of the POLYGON in each grid cell
            overlaps[ii,jj] = overlap_cell.area/fire_shape.area
    
    return overlaps

In [54]:
lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon)
calculate_overlaps(lat, lon, lat_corners, lon_corners)

ctr 31.5 -124.375
SW 31.25 -124.6875
SE 31.25 -124.0625
NW 31.75 -124.6875
NE 31.75 -124.0625
ctr 31.5 -123.75
SW 31.25 -124.0625
SE 31.25 -123.4375
NW 31.75 -124.0625
NE 31.75 -123.4375
ctr 31.5 -123.125
SW 31.25 -123.4375
SE 31.25 -122.8125
NW 31.75 -123.4375
NE 31.75 -122.8125
ctr 31.5 -122.5
SW 31.25 -122.8125
SE 31.25 -122.1875
NW 31.75 -122.8125
NE 31.75 -122.1875
ctr 31.5 -121.875
SW 31.25 -122.1875
SE 31.25 -121.5625
NW 31.75 -122.1875
NE 31.75 -121.5625
ctr 31.5 -121.25
SW 31.25 -121.5625
SE 31.25 -120.9375
NW 31.75 -121.5625
NE 31.75 -120.9375
ctr 31.5 -120.625
SW 31.25 -120.9375
SE 31.25 -120.3125
NW 31.75 -120.9375
NE 31.75 -120.3125
ctr 31.5 -120.0
SW 31.25 -120.3125
SE 31.25 -119.6875
NW 31.75 -120.3125
NE 31.75 -119.6875
ctr 31.5 -119.375
SW 31.25 -119.6875
SE 31.25 -119.0625
NW 31.75 -119.6875
NE 31.75 -119.0625
ctr 31.5 -118.75
SW 31.25 -119.0625
SE 31.25 -118.4375
NW 31.75 -119.0625
NE 31.75 -118.4375
ctr 31.5 -118.125
SW 31.25 -118.4375
SE 31.25 -117.8125
NW 31.75 -1

SW 33.25 -109.0625
SE 33.25 -108.4375
NW 33.75 -109.0625
NE 33.75 -108.4375
ctr 33.5 -108.125
SW 33.25 -108.4375
SE 33.25 -107.8125
NW 33.75 -108.4375
NE 33.75 -107.8125
ctr 33.5 -107.5
SW 33.25 -107.8125
SE 33.25 -107.1875
NW 33.75 -107.8125
NE 33.75 -107.1875
ctr 33.5 -106.875
SW 33.25 -107.1875
SE 33.25 -106.5625
NW 33.75 -107.1875
NE 33.75 -106.5625
ctr 33.5 -106.25
SW 33.25 -106.5625
SE 33.25 -105.9375
NW 33.75 -106.5625
NE 33.75 -105.9375
ctr 33.5 -105.625
SW 33.25 -105.9375
SE 33.25 -105.3125
NW 33.75 -105.9375
NE 33.75 -105.3125
ctr 33.5 -105.0
SW 33.25 -105.3125
SE 33.25 -104.6875
NW 33.75 -105.3125
NE 33.75 -104.6875
ctr 33.5 -104.375
SW 33.25 -104.6875
SE 33.25 -104.0625
NW 33.75 -104.6875
NE 33.75 -104.0625
ctr 33.5 -103.75
SW 33.25 -104.0625
SE 33.25 -103.4375
NW 33.75 -104.0625
NE 33.75 -103.4375
ctr 33.5 -103.125
SW 33.25 -103.4375
SE 33.25 -102.8125
NW 33.75 -103.4375
NE 33.75 -102.8125
ctr 33.5 -102.5
SW 33.25 -102.8125
SE 33.25 -102.1875
NW 33.75 -102.8125
NE 33.75 -1

SE 36.75 -123.4375
NW 37.25 -124.0625
NE 37.25 -123.4375
ctr 37.0 -123.125
SW 36.75 -123.4375
SE 36.75 -122.8125
NW 37.25 -123.4375
NE 37.25 -122.8125
ctr 37.0 -122.5
SW 36.75 -122.8125
SE 36.75 -122.1875
NW 37.25 -122.8125
NE 37.25 -122.1875
ctr 37.0 -121.875
SW 36.75 -122.1875
SE 36.75 -121.5625
NW 37.25 -122.1875
NE 37.25 -121.5625
ctr 37.0 -121.25
SW 36.75 -121.5625
SE 36.75 -120.9375
NW 37.25 -121.5625
NE 37.25 -120.9375
ctr 37.0 -120.625
SW 36.75 -120.9375
SE 36.75 -120.3125
NW 37.25 -120.9375
NE 37.25 -120.3125
ctr 37.0 -120.0
SW 36.75 -120.3125
SE 36.75 -119.6875
NW 37.25 -120.3125
NE 37.25 -119.6875
ctr 37.0 -119.375
SW 36.75 -119.6875
SE 36.75 -119.0625
NW 37.25 -119.6875
NE 37.25 -119.0625
ctr 37.0 -118.75
SW 36.75 -119.0625
SE 36.75 -118.4375
NW 37.25 -119.0625
NE 37.25 -118.4375
ctr 37.0 -118.125
SW 36.75 -118.4375
SE 36.75 -117.8125
NW 37.25 -118.4375
NE 37.25 -117.8125
ctr 37.0 -117.5
SW 36.75 -117.8125
SE 36.75 -117.1875
NW 37.25 -117.8125
NE 37.25 -117.1875
ctr 37.0 -1

SE 40.75 -120.9375
NW 41.25 -121.5625
NE 41.25 -120.9375
ctr 41.0 -120.625
SW 40.75 -120.9375
SE 40.75 -120.3125
NW 41.25 -120.9375
NE 41.25 -120.3125
ctr 41.0 -120.0
SW 40.75 -120.3125
SE 40.75 -119.6875
NW 41.25 -120.3125
NE 41.25 -119.6875
ctr 41.0 -119.375
SW 40.75 -119.6875
SE 40.75 -119.0625
NW 41.25 -119.6875
NE 41.25 -119.0625
ctr 41.0 -118.75
SW 40.75 -119.0625
SE 40.75 -118.4375
NW 41.25 -119.0625
NE 41.25 -118.4375
ctr 41.0 -118.125
SW 40.75 -118.4375
SE 40.75 -117.8125
NW 41.25 -118.4375
NE 41.25 -117.8125
ctr 41.0 -117.5
SW 40.75 -117.8125
SE 40.75 -117.1875
NW 41.25 -117.8125
NE 41.25 -117.1875
ctr 41.0 -116.875
SW 40.75 -117.1875
SE 40.75 -116.5625
NW 41.25 -117.1875
NE 41.25 -116.5625
ctr 41.0 -116.25
SW 40.75 -116.5625
SE 40.75 -115.9375
NW 41.25 -116.5625
NE 41.25 -115.9375
ctr 41.0 -115.625
SW 40.75 -115.9375
SE 40.75 -115.3125
NW 41.25 -115.9375
NE 41.25 -115.3125
ctr 41.0 -115.0
SW 40.75 -115.3125
SE 40.75 -114.6875
NW 41.25 -115.3125
NE 41.25 -114.6875
ctr 41.0 -1

NW 44.25 -114.6875
NE 44.25 -114.0625
ctr 44.0 -113.75
SW 43.75 -114.0625
SE 43.75 -113.4375
NW 44.25 -114.0625
NE 44.25 -113.4375
ctr 44.0 -113.125
SW 43.75 -113.4375
SE 43.75 -112.8125
NW 44.25 -113.4375
NE 44.25 -112.8125
ctr 44.0 -112.5
SW 43.75 -112.8125
SE 43.75 -112.1875
NW 44.25 -112.8125
NE 44.25 -112.1875
ctr 44.0 -111.875
SW 43.75 -112.1875
SE 43.75 -111.5625
NW 44.25 -112.1875
NE 44.25 -111.5625
ctr 44.0 -111.25
SW 43.75 -111.5625
SE 43.75 -110.9375
NW 44.25 -111.5625
NE 44.25 -110.9375
ctr 44.0 -110.625
SW 43.75 -110.9375
SE 43.75 -110.3125
NW 44.25 -110.9375
NE 44.25 -110.3125
ctr 44.0 -110.0
SW 43.75 -110.3125
SE 43.75 -109.6875
NW 44.25 -110.3125
NE 44.25 -109.6875
ctr 44.0 -109.375
SW 43.75 -109.6875
SE 43.75 -109.0625
NW 44.25 -109.6875
NE 44.25 -109.0625
ctr 44.0 -108.75
SW 43.75 -109.0625
SE 43.75 -108.4375
NW 44.25 -109.0625
NE 44.25 -108.4375
ctr 44.0 -108.125
SW 43.75 -108.4375
SE 43.75 -107.8125
NW 44.25 -108.4375
NE 44.25 -107.8125
ctr 44.0 -107.5
SW 43.75 -107

SE 46.75 -109.0625
NW 47.25 -109.6875
NE 47.25 -109.0625
ctr 47.0 -108.75
SW 46.75 -109.0625
SE 46.75 -108.4375
NW 47.25 -109.0625
NE 47.25 -108.4375
ctr 47.0 -108.125
SW 46.75 -108.4375
SE 46.75 -107.8125
NW 47.25 -108.4375
NE 47.25 -107.8125
ctr 47.0 -107.5
SW 46.75 -107.8125
SE 46.75 -107.1875
NW 47.25 -107.8125
NE 47.25 -107.1875
ctr 47.0 -106.875
SW 46.75 -107.1875
SE 46.75 -106.5625
NW 47.25 -107.1875
NE 47.25 -106.5625
ctr 47.0 -106.25
SW 46.75 -106.5625
SE 46.75 -105.9375
NW 47.25 -106.5625
NE 47.25 -105.9375
ctr 47.0 -105.625
SW 46.75 -105.9375
SE 46.75 -105.3125
NW 47.25 -105.9375
NE 47.25 -105.3125
ctr 47.0 -105.0
SW 46.75 -105.3125
SE 46.75 -104.6875
NW 47.25 -105.3125
NE 47.25 -104.6875
ctr 47.0 -104.375
SW 46.75 -104.6875
SE 46.75 -104.0625
NW 47.25 -104.6875
NE 47.25 -104.0625
ctr 47.0 -103.75
SW 46.75 -104.0625
SE 46.75 -103.4375
NW 47.25 -104.0625
NE 47.25 -103.4375
ctr 47.0 -103.125
SW 46.75 -103.4375
SE 46.75 -102.8125
NW 47.25 -103.4375
NE 47.25 -102.8125
ctr 47.0 -

In [144]:
#input: the variable vector. Should be the names of the lat and lon variables
def extract_dimensions_2d(varlist, dat):
    lat = dat[varlist[0]][:].data
    lon = dat[varlist[1]][:].data
    if (len(lat.shape)==1)&(len(lon.shape)==1): # if the lat and lons are vectors
        lon, lat = np.meshgrid(lon, lat)
    return lon, lat

In [165]:
#input: the variable vector. Should be the names of the lat, lon, and time variables
def extract_dimensions_3d(varlist, dat):
    lat = dat[varlist[0]][:].data
    lon = dat[varlist[1]][:].data
    time = dat[varlist[2]][:].data
    if (len(lat.shape)==1)&(len(lon.shape)==1)&(len(time.shape)==1): # if the lat and lons are vectors
        time, lon, lat = np.meshgrid(lon, lat, time)
    return time, lon, lat

In [ ]:
def read_dimensions

In [6]:
varis = {'/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.':['TLML', 'QLML', 'SPEEDLML', 'PS']}
dims = {'/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.':['time', 'lat', 'lon']}
file_suffix = '.nc4'


for i in varis : #loop over the file names
    for k in dims[i]: #loop over the dimensions and set them up!
        #setting up the dimensions means putting the lat and lon on a grid and 
        print(k)
    for j in varis[i]: #loop over the variable names, which are lists
        print(i,j)

time
lat
lon
/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx. TLML
/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx. QLML
/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx. SPEEDLML
/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx. PS


In [67]:
extract_variables_2d(dat)


<class 'netCDF4._netCDF4.Variable'>
float32 PS(time, lat, lon)
    long_name: surface_pressure
    units: Pa
    _FillValue: 1000000000000000.0
    missing_value: 1000000000000000.0
    fmissing_value: 1000000000000000.0
    scale_factor: 1.0
    add_offset: 0.0
    standard_name: surface_pressure
    vmax: 1000000000000000.0
    vmin: -1000000000000000.0
    valid_range: [-1.e+15  1.e+15]
unlimited dimensions: time
current shape = (24, 37, 39)
filling on
<class 'netCDF4._netCDF4.Variable'>
float32 QLML(time, lat, lon)
    long_name: surface_specific_humidity
    units: 1
    _FillValue: 1000000000000000.0
    missing_value: 1000000000000000.0
    fmissing_value: 1000000000000000.0
    scale_factor: 1.0
    add_offset: 0.0
    standard_name: surface_specific_humidity
    vmax: 1000000000000000.0
    vmin: -1000000000000000.0
    valid_range: [-1.e+15  1.e+15]
unlimited dimensions: time
current shape = (24, 37, 39)
filling on
<class 'netCDF4._netCDF4.Variable'>
float32 SPEEDLML(time, la